# Dataset 2018-2019 Bereinigung und Normalisierung
## Spezialisiertes Modul für Kaggle/Immobilienscout24 Dataset

### Ziel
Bereinigung und Normalisierung des historischen Datasets (2018-2019) in ein standardisiertes Format für die gemeinsame Analyse.

### Input
- `data/raw/Dataset_2018_2019.csv`

### Output
- `data/processed/dataset_2018_2019_normalized.csv`

### Standardisierte Ausgabespalten
- `price`: Normalisierter Preis (Kaltmiete in €)
- `size`: Normalisierte Größe (m²)
- `district`: Berliner Bezirk (standardisiert)
- `rooms`: Anzahl Zimmer
- `year`: Jahr des Datasets (2019)
- `dataset_id`: Eindeutige Dataset-Kennzeichnung (historical)
- `source`: Datenquelle

---
**Teil der modularen Preprocessing-Pipeline**  
**Datum:** 4. Juli 2025  
**Version:** 1.0

## 1. Import Required Libraries

In [11]:
# Import required libraries
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

# Display configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 20)

print("Bibliotheken erfolgreich importiert!")
print(f"Pandas Version: {pd.__version__}")
print(f"Dataset: 2018-2019 (Kaggle/Immobilienscout24)")

Bibliotheken erfolgreich importiert!
Pandas Version: 2.3.0
Dataset: 2018-2019 (Kaggle/Immobilienscout24)


## 2. Daten laden und erste Analyse

In [2]:
# Lade Dataset 2018-2019
print("=" * 60)
print("DATASET 2018-2019 LADEN UND ANALYSIEREN")
print("=" * 60)

# Lade Rohdaten
df_raw = pd.read_csv('data/raw/Dataset_2018_2019.csv')
print(f"Dataset geladen: {df_raw.shape[0]:,} Zeilen, {df_raw.shape[1]} Spalten")

# Grundlegende Informationen
print(f"\nSpalten: {list(df_raw.columns)}")
print(f"\nDatentypen:")
print(df_raw.dtypes)

# Fehlende Werte
print(f"\nFehlende Werte:")
missing_values = df_raw.isnull().sum()
missing_pct = (missing_values / len(df_raw) * 100).round(2)
for col in missing_values[missing_values > 0].index:
    print(f"  {col}: {missing_values[col]} ({missing_pct[col]}%)")

# Erste 5 Zeilen
print(f"\nErste 5 Zeilen:")
print(df_raw.head())

DATASET 2018-2019 LADEN UND ANALYSIEREN
Dataset geladen: 10,406 Zeilen, 9 Spalten

Spalten: ['regio3', 'street', 'livingSpace', 'baseRent', 'totalRent', 'noRooms', 'floor', 'typeOfFlat', 'yearConstructed']

Datentypen:
regio3              object
street              object
livingSpace        float64
baseRent           float64
totalRent          float64
noRooms            float64
floor              float64
typeOfFlat          object
yearConstructed    float64
dtype: object

Fehlende Werte:
  totalRent: 662 (6.36%)
  floor: 1100 (10.57%)
  typeOfFlat: 804 (7.73%)
  yearConstructed: 1425 (13.69%)

Erste 5 Zeilen:
            regio3                      street  livingSpace  baseRent  totalRent  noRooms  floor    typeOfFlat  yearConstructed
0  Staaken_Spandau           Metropolitan Park        77.00    820.00    1140.00      3.0    0.0  ground_floor              NaN
1        Weißensee      B&ouml;rnestra&szlig;e        62.63    808.00     955.00      2.0    0.0  ground_floor           1918.0

## 3. Spezifische Bereinigung Dataset 2018-2019

In [3]:
# Spezifische Bereinigung für Dataset 2018-2019
print("=" * 60)
print("SPEZIFISCHE BEREINIGUNG DATASET 2018-2019")
print("=" * 60)

# Erstelle Arbeitskopie
df = df_raw.copy()
print(f"Arbeitskopie erstellt: {len(df)} Zeilen")

# 1. Preis-Bereinigung (baseRent)
print(f"\n=== PREIS-BEREINIGUNG ===")
print(f"baseRent - Statistik vor Bereinigung:")
print(f"  Typ: {df['baseRent'].dtype}")
print(f"  Nicht-null Werte: {df['baseRent'].notna().sum()}")
print(f"  Min: {df['baseRent'].min()}, Max: {df['baseRent'].max()}")

# Preis ist bereits numerisch, nur Plausibilitätsprüfung
# Entferne unrealistische Preise (< 100€ oder > 10.000€)
original_count = len(df)
df = df[(df['baseRent'] >= 100) & (df['baseRent'] <= 10000)]
removed_price = original_count - len(df)
print(f"Entfernte unrealistische Preise: {removed_price}")

# 2. Größen-Bereinigung (livingSpace)
print(f"\n=== GRÖSSEN-BEREINIGUNG ===")
print(f"livingSpace - Statistik vor Bereinigung:")
print(f"  Typ: {df['livingSpace'].dtype}")
print(f"  Nicht-null Werte: {df['livingSpace'].notna().sum()}")
print(f"  Min: {df['livingSpace'].min()}, Max: {df['livingSpace'].max()}")

# Größe ist bereits numerisch, nur Plausibilitätsprüfung
# Entferne unrealistische Größen (< 10m² oder > 500m²)
original_count = len(df)
df = df[(df['livingSpace'] >= 10) & (df['livingSpace'] <= 500)]
removed_size = original_count - len(df)
print(f"Entfernte unrealistische Größen: {removed_size}")

# 3. Bezirks-Normalisierung (regio3)
print(f"\n=== BEZIRKS-NORMALISIERUNG ===")
print(f"regio3 - Einzigartige Werte: {df['regio3'].nunique()}")
print(f"Bezirke: {sorted(df['regio3'].unique())}")

# Bezirk-Normalisierung (entferne _Suffix)
def normalize_district_2018_2019(district):
    """Normalisiert Bezirksnamen für Dataset 2018-2019"""
    if pd.isna(district):
        return None
    
    # Entferne Suffix nach Unterstrich
    if '_' in str(district):
        return str(district).split('_')[0]
    
    return str(district)

df['district_normalized'] = df['regio3'].apply(normalize_district_2018_2019)

print(f"Normalisierte Bezirke: {sorted(df['district_normalized'].unique())}")
print(f"Anzahl normalisierte Bezirke: {df['district_normalized'].nunique()}")

# 4. Zimmer-Bereinigung (noRooms)
print(f"\n=== ZIMMER-BEREINIGUNG ===")
print(f"noRooms - Statistik:")
print(f"  Typ: {df['noRooms'].dtype}")
print(f"  Nicht-null Werte: {df['noRooms'].notna().sum()}")
print(f"  Einzigartige Werte: {sorted(df['noRooms'].dropna().unique())}")

# Zimmeranzahl ist bereits numerisch
# Plausibilitätsprüfung (0.5 bis 10 Zimmer)
original_count = len(df)
df = df[(df['noRooms'] >= 0.5) & (df['noRooms'] <= 10)]
removed_rooms = original_count - len(df)
print(f"Entfernte unrealistische Zimmeranzahlen: {removed_rooms}")

print(f"Spezifische Bereinigung abgeschlossen")
print(f"Verbleibende Datensätze: {len(df)} (Verlust: {len(df_raw) - len(df)})")

SPEZIFISCHE BEREINIGUNG DATASET 2018-2019
Arbeitskopie erstellt: 10406 Zeilen

=== PREIS-BEREINIGUNG ===
baseRent - Statistik vor Bereinigung:
  Typ: float64
  Nicht-null Werte: 10406
  Min: 0.0, Max: 20000.0
Entfernte unrealistische Preise: 11

=== GRÖSSEN-BEREINIGUNG ===
livingSpace - Statistik vor Bereinigung:
  Typ: float64
  Nicht-null Werte: 10395
  Min: 3.0, Max: 542.53
Entfernte unrealistische Größen: 7

=== BEZIRKS-NORMALISIERUNG ===
regio3 - Einzigartige Werte: 79
Bezirke: ['Adlershof_Treptow', 'Alt_Hohenschönhausen_Hohenschönhausen', 'Altglienicke_Treptow', 'Baumschulenweg_Treptow', 'Biesdorf_Marzahn', 'Blankenburg_Weißensee', 'Bohnsdorf_Treptow', 'Britz_Neukölln', 'Buch_Pankow', 'Buckow_Neukölln', 'Charlottenburg', 'Dahlem_Zehlendorf', 'Falkenberg_Hohenschönhausen', 'Französisch_Buchholz_Pankow', 'Friedenau_Schöneberg', 'Friedrichsfelde_Lichtenberg', 'Friedrichshagen_Köpenick', 'Friedrichshain', 'Frohnau_Reinickendorf', 'Gatow_Spandau', 'Grunewald_Wilmersdorf', 'Grünau_Köpe

## 4. Normalisierung in Standardformat

In [4]:
# Normalisierung in Standardformat
print("=" * 60)
print("NORMALISIERUNG IN STANDARDFORMAT")
print("=" * 60)

# Erstelle normalisiertes Dataset mit Standardspalten
df_normalized = pd.DataFrame()

# Standardspalten zuweisen
df_normalized['price'] = df['baseRent'].astype('float64')
df_normalized['size'] = df['livingSpace'].astype('float64')
df_normalized['district'] = df['district_normalized'].astype('string')
df_normalized['rooms'] = df['noRooms'].astype('float64')
df_normalized['year'] = 2019
df_normalized['dataset_id'] = 'historical'
df_normalized['source'] = 'Kaggle/Immobilienscout24'

# Zusätzliche Spalten aus Original-Dataset beibehalten
df_normalized['street'] = df['street']
df_normalized['floor'] = df['floor']
df_normalized['typeOfFlat'] = df['typeOfFlat']
df_normalized['yearConstructed'] = df['yearConstructed']
df_normalized['totalRent'] = df['totalRent']

print(f"Normalisiertes Dataset erstellt: {len(df_normalized)} Zeilen")
print(f"Standardspalten: {['price', 'size', 'district', 'rooms', 'year', 'dataset_id', 'source']}")
print(f"Zusätzliche Spalten: {list(df_normalized.columns[7:])}")

# Datenqualität prüfen
print(f"=== DATENQUALITÄT NORMALISIERTES DATASET ===")
print(f"Zeilen mit Preis: {df_normalized['price'].notna().sum()}")
print(f"Zeilen mit Größe: {df_normalized['size'].notna().sum()}")
print(f"Zeilen mit Bezirk: {df_normalized['district'].notna().sum()}")
print(f"Zeilen mit Zimmeranzahl: {df_normalized['rooms'].notna().sum()}")

# Statistiken
print(f"=== STATISTIKEN ===")
print(f"Preis - Min: {df_normalized['price'].min():.2f}€, Max: {df_normalized['price'].max():.2f}€, Median: {df_normalized['price'].median():.2f}€")
print(f"Größe - Min: {df_normalized['size'].min():.1f}m², Max: {df_normalized['size'].max():.1f}m², Median: {df_normalized['size'].median():.1f}m²")
print(f"Zimmer - Min: {df_normalized['rooms'].min():.1f}, Max: {df_normalized['rooms'].max():.1f}, Median: {df_normalized['rooms'].median():.1f}")

# Bezirksverteilung
print(f"=== BEZIRKSVERTEILUNG ===")
district_counts = df_normalized['district'].value_counts()
print(f"Anzahl Bezirke: {len(district_counts)}")
for district, count in district_counts.head(10).items():
    print(f"  {district}: {count} Einträge")

print(f"Normalisierung abgeschlossen!")

NORMALISIERUNG IN STANDARDFORMAT
Normalisiertes Dataset erstellt: 10387 Zeilen
Standardspalten: ['price', 'size', 'district', 'rooms', 'year', 'dataset_id', 'source']
Zusätzliche Spalten: ['street', 'floor', 'typeOfFlat', 'yearConstructed', 'totalRent']
=== DATENQUALITÄT NORMALISIERTES DATASET ===
Zeilen mit Preis: 10387
Zeilen mit Größe: 10387
Zeilen mit Bezirk: 10387
Zeilen mit Zimmeranzahl: 10387
=== STATISTIKEN ===
Preis - Min: 178.16€, Max: 9500.00€, Median: 945.00€
Größe - Min: 10.0m², Max: 482.0m², Median: 72.0m²
Zimmer - Min: 1.0, Max: 10.0, Median: 2.0
=== BEZIRKSVERTEILUNG ===
Anzahl Bezirke: 79
  Mitte: 799 Einträge
  Tiergarten: 768 Einträge
  Charlottenburg: 701 Einträge
  Friedrichshain: 553 Einträge
  Prenzlauer: 473 Einträge
  Spandau: 415 Einträge
  Wedding: 397 Einträge
  Wilmersdorf: 370 Einträge
  Neukölln: 361 Einträge
  Köpenick: 351 Einträge
Normalisierung abgeschlossen!


## 5. Export des normalisierten Datasets

In [5]:
# Export des normalisierten Datasets
print("=" * 60)
print("EXPORT NORMALISIERTES DATASET")
print("=" * 60)

# Ausgabedatei
output_file = 'data/processed/dataset_2018_2019_normalized.csv'

# Export
df_normalized.to_csv(output_file, index=False)
print(f"✅ Normalisiertes Dataset exportiert: {output_file}")
print(f"Dateigröße: {len(df_normalized)} Zeilen x {len(df_normalized.columns)} Spalten")

# Validierung des Exports
test_load = pd.read_csv(output_file)
print(f"✅ Export-Validierung erfolgreich: {len(test_load)} Zeilen geladen")

# Zusammenfassung
print(f"=== ZUSAMMENFASSUNG DATASET 2018-2019 ===")
print(f"Input: data/raw/Dataset_2018_2019.csv ({len(df_raw)} Zeilen)")
print(f"Output: {output_file} ({len(df_normalized)} Zeilen)")
print(f"Datenverlust: {len(df_raw) - len(df_normalized)} Zeilen ({((len(df_raw) - len(df_normalized))/len(df_raw)*100):.1f}%)")
print(f"Standardisierte Spalten: price, size, district, rooms, year, dataset_id, source")
print(f"Zusätzliche Spalten: {len(df_normalized.columns) - 7}")

print(f"🎯 DATASET 2018-2019 BEREINIGUNG ABGESCHLOSSEN!")
print(f"Bereit für Kombination mit anderen normalisierten Datasets.")

EXPORT NORMALISIERTES DATASET
✅ Normalisiertes Dataset exportiert: data/processed/dataset_2018_2019_normalized.csv
Dateigröße: 10387 Zeilen x 12 Spalten
✅ Export-Validierung erfolgreich: 10387 Zeilen geladen
=== ZUSAMMENFASSUNG DATASET 2018-2019 ===
Input: data/raw/Dataset_2018_2019.csv (10406 Zeilen)
Output: data/processed/dataset_2018_2019_normalized.csv (10387 Zeilen)
Datenverlust: 19 Zeilen (0.2%)
Standardisierte Spalten: price, size, district, rooms, year, dataset_id, source
Zusätzliche Spalten: 5
🎯 DATASET 2018-2019 BEREINIGUNG ABGESCHLOSSEN!
Bereit für Kombination mit anderen normalisierten Datasets.


## 6. Lade angereicherte Wohnlagendaten

In [6]:
print("="*60)
print("ANGEREICHERTE WOHNLAGENDATEN LADEN")
print("="*60)

enriched_data_path = 'data/raw/wohnlagen_enriched.csv'
try:
    enriched_df = pd.read_csv(enriched_data_path)
    print(f"✅ Angereicherte Daten geladen: {len(enriched_df):,} Zeilen, {len(enriched_df.columns)} Spalten")
except FileNotFoundError:
    print(f"❌ Datei nicht gefunden: {enriched_data_path}")

ANGEREICHERTE WOHNLAGENDATEN LADEN
✅ Angereicherte Daten geladen: 551,249 Zeilen, 11 Spalten
✅ Angereicherte Daten geladen: 551,249 Zeilen, 11 Spalten


## 7. Kombiniere Datasets mit Wohnlagendaten

In [16]:
print("="*60)
print("KOMBINIERE MIT WOHNLAGENDATEN")
print("="*60)

# Debug: Check original data sizes
print(f"Original df_normalized: {len(df_normalized):,} Zeilen")
print(f"Original enriched_df: {len(enriched_df):,} Zeilen")

# Stelle sicher, dass PLZ-Spalte vorhanden ist
if 'plz' not in df_normalized.columns:
    print("❌ PLZ-Spalte fehlt! Führe PLZ-Extraktion zuerst aus.")
    df_normalized['plz'] = None
else:
    plz_available = df_normalized['plz'].notna().sum()
    print(f"PLZ verfügbar: {plz_available:,}/{len(df_normalized):,} ({plz_available/len(df_normalized)*100:.1f}%)")

# Create a unique mapping to avoid cartesian product - use only unique street names
enriched_df_subset = enriched_df[['strasse', 'wol', 'ortsteil_neu']].drop_duplicates(subset=['strasse'])
print(f"Unique street mappings: {len(enriched_df_subset):,} Zeilen")

# Debug: Check street overlap
df_streets_unique = set(df_normalized['street'].unique())
enriched_streets_unique = set(enriched_df_subset['strasse'].unique())
overlap = df_streets_unique.intersection(enriched_streets_unique)
print(f"Street overlap: {len(overlap)} von {len(df_streets_unique)} Straßen im Dataset")

# Perform the merge (mit PLZ)
df_enriched = pd.merge(df_normalized, enriched_df_subset, how='left', left_on=['street'], right_on=['strasse'])

# Bereinige überflüssige Spalten
if 'strasse' in df_enriched.columns:
    df_enriched = df_enriched.drop('strasse', axis=1)

print(f"✅ Kombiniertes und angereichertes Dataset erstellt: {len(df_enriched):,} Zeilen")

# Check merge success
successful_enrichment = df_enriched['ortsteil_neu'].notna().sum()
print(f"Erfolgreiche Anreicherung: {successful_enrichment:,} von {len(df_enriched):,} Zeilen ({successful_enrichment/len(df_enriched)*100:.1f}%)")

# Finale Spalten-Zusammenfassung
print(f"\nFinale Spalten im angereicherten Dataset:")
print(f"  {list(df_enriched.columns)}")

# PLZ-Qualität prüfen
if 'plz' in df_enriched.columns:
    final_plz_count = df_enriched['plz'].notna().sum()
    print(f"PLZ im finalen Dataset: {final_plz_count:,}/{len(df_enriched):,} ({final_plz_count/len(df_enriched)*100:.1f}%)")
    
    if final_plz_count > 0:
        print("Top 5 PLZ im finalen Dataset:")
        for plz, count in df_enriched['plz'].value_counts().head(5).items():
            print(f"  {plz}: {count:,} Einträge")

KOMBINIERE MIT WOHNLAGENDATEN
Original df_normalized: 10,387 Zeilen
Original enriched_df: 551,249 Zeilen
PLZ verfügbar: 9,505/10,387 (91.5%)
Unique street mappings: 9,479 Zeilen
Street overlap: 533 von 3267 Straßen im Dataset
✅ Kombiniertes und angereichertes Dataset erstellt: 10,387 Zeilen
Erfolgreiche Anreicherung: 1,760 von 10,387 Zeilen (16.9%)

Finale Spalten im angereicherten Dataset:
  ['price', 'size', 'district', 'rooms', 'year', 'dataset_id', 'source', 'street', 'floor', 'typeOfFlat', 'yearConstructed', 'totalRent', 'plz', 'wol', 'ortsteil_neu']
PLZ im finalen Dataset: 9,505/10,387 (91.5%)
Top 5 PLZ im finalen Dataset:
  10179: 762 Einträge
  10557: 721 Einträge
  14055: 567 Einträge
  12555: 498 Einträge
  10247: 424 Einträge


## 7.5. PLZ-Extraktion für 2018-2019 Dataset

**🎯 Problem:** Das 2018-2019 Dataset enthält keine PLZ-Spalte, nur `regio3` (Ortsteil) und `street`.

**🔧 Lösung:** Wir extrahieren die PLZ durch Matching mit der `wohnlagen_enriched.csv`:
1. **Strategie 1:** Ortsteil-basiert (regio3 → PLZ)
2. **Strategie 2:** Street-basiert (street → PLZ) 
3. **Fallback:** Bezirk-basiert

Dies ist essentiell für die spätere PLZ-Enhancement-Pipeline und räumliche Genauigkeit.

In [17]:
print("="*60)
print("PLZ-EXTRAKTION FÜR 2018-2019 DATASET")
print("="*60)

# Prüfe verfügbare Spalten
print(f"df_normalized Spalten: {list(df_normalized.columns)}")
print(f"df_raw Spalten: {list(df_raw.columns)}")

# Da regio3 nicht in df_normalized ist, hole es aus df_raw
# Erstelle einen temporären DataFrame für die PLZ-Extraktion
temp_df = df_normalized.copy()

# Füge regio3 aus df_raw hinzu (basierend auf Index)
if len(temp_df) == len(df_raw):
    temp_df['regio3'] = df_raw['regio3'].values
    print(f"✅ regio3 Spalte hinzugefügt: {temp_df['regio3'].nunique()} einzigartige Werte")
else:
    print(f"❌ Längen-Mismatch: df_normalized={len(temp_df)}, df_raw={len(df_raw)}")

# Strategie 1: Ortsteil-basierte PLZ-Extraktion
print("\n=== STRATEGIE 1: ORTSTEIL-BASIERTE PLZ-EXTRAKTION ===")

# Erstelle Ortsteil-zu-PLZ-Mapping aus wohnlagen_enriched.csv
ortsteil_plz_mapping = {}
if 'enriched_df' in locals():
    # Extrahiere einzigartige Ortsteil-PLZ-Kombinationen
    ortsteil_plz_pairs = enriched_df[['ortsteil_neu', 'plz']].dropna().drop_duplicates()
    
    # Für Ortsteile mit mehreren PLZ, nehme die häufigste
    for ortsteil in ortsteil_plz_pairs['ortsteil_neu'].unique():
        plz_counts = enriched_df[enriched_df['ortsteil_neu'] == ortsteil]['plz'].value_counts()
        if len(plz_counts) > 0:
            most_common_plz = plz_counts.index[0]
            ortsteil_plz_mapping[ortsteil] = most_common_plz
    
    print(f"Ortsteil-PLZ-Mapping erstellt: {len(ortsteil_plz_mapping)} Einträge")
    
    # Zeige einige Beispiele
    print("Beispiele:")
    for ortsteil, plz in list(ortsteil_plz_mapping.items())[:5]:
        print(f"  {ortsteil} → {plz}")

# Strategie 2: Street-basierte PLZ-Extraktion
print("\n=== STRATEGIE 2: STREET-BASIERTE PLZ-EXTRAKTION ===")

# Erstelle Street-zu-PLZ-Mapping
street_plz_mapping = {}
if 'enriched_df' in locals():
    # Extrahiere einzigartige Street-PLZ-Kombinationen
    street_plz_pairs = enriched_df[['strasse', 'plz']].dropna().drop_duplicates()
    
    # Für Straßen mit mehreren PLZ, nehme die häufigste
    for street in street_plz_pairs['strasse'].unique():
        plz_counts = enriched_df[enriched_df['strasse'] == street]['plz'].value_counts()
        if len(plz_counts) > 0:
            most_common_plz = plz_counts.index[0]
            street_plz_mapping[street] = most_common_plz
    
    print(f"Street-PLZ-Mapping erstellt: {len(street_plz_mapping)} Einträge")
    
    # Zeige einige Beispiele
    print("Beispiele:")
    for street, plz in list(street_plz_mapping.items())[:5]:
        print(f"  {street} → {plz}")

# Normalisiere regio3 für besseres Matching
print("\n=== REGIO3-NORMALISIERUNG ===")

def normalize_regio3(regio3_value):
    """Normalisiert regio3 Werte für besseres Ortsteil-Matching."""
    if pd.isna(regio3_value):
        return None
    
    # Konvertiere zu String und bereinige
    normalized = str(regio3_value).strip()
    
    # Entferne Bezirk-Suffix (z.B. "Prenzlauer_Berg_Prenzlauer_Berg" → "Prenzlauer Berg")
    if '_' in normalized:
        parts = normalized.split('_')
        # Nehme den ersten Teil oder entferne Duplikate
        if len(parts) == 2 and parts[0] == parts[1]:
            normalized = parts[0]
        else:
            normalized = parts[0]
    
    # Ersetze Underscores durch Spaces
    normalized = normalized.replace('_', ' ')
    
    # Spezielle Mappings für bekannte Varianten
    mappings = {
        'Neu Hohenschönhausen': 'Neu-Hohenschönhausen',
        'Hohenschönhausen': 'Alt-Hohenschönhausen',
        'Französisch Buchholz': 'Französisch Buchholz',
        'Grünau': 'Grünau',
        'Köpenick': 'Köpenick'
    }
    
    return mappings.get(normalized, normalized)

# Teste die Normalisierung (verwende temp_df mit regio3)
if 'regio3' in temp_df.columns:
    temp_df['regio3_normalized'] = temp_df['regio3'].apply(normalize_regio3)
    
    print("Regio3 Normalisierung - Beispiele:")
    regio3_examples = temp_df[['regio3', 'regio3_normalized']].drop_duplicates().head(10)
    for _, row in regio3_examples.iterrows():
        print(f"  {row['regio3']} → {row['regio3_normalized']}")
    
    print(f"\nUnikat regio3 (original): {temp_df['regio3'].nunique()}")
    print(f"Unikat regio3 (normalized): {temp_df['regio3_normalized'].nunique()}")
else:
    print("❌ regio3 Spalte nicht verfügbar")

# Wende PLZ-Extraktion an
print("\n=== PLZ-EXTRAKTION ANWENDEN ===")

# Initialisiere PLZ-Spalte
temp_df['plz'] = None

# Strategie 1: Ortsteil-basiert
ortsteil_matches = 0
if 'regio3_normalized' in temp_df.columns:
    for idx, row in temp_df.iterrows():
        normalized_ortsteil = row['regio3_normalized']
        if normalized_ortsteil in ortsteil_plz_mapping:
            temp_df.loc[idx, 'plz'] = ortsteil_plz_mapping[normalized_ortsteil]
            ortsteil_matches += 1

# Strategie 2: Street-basiert (für fehlende PLZ)
street_matches = 0
for idx, row in temp_df.iterrows():
    if pd.isna(row['plz']) and row['street'] in street_plz_mapping:
        temp_df.loc[idx, 'plz'] = street_plz_mapping[row['street']]
        street_matches += 1

# Übertrage PLZ zurück zu df_normalized
df_normalized['plz'] = temp_df['plz']

# Ergebnisse
total_with_plz = df_normalized['plz'].notna().sum()
print(f"PLZ-Extraktion Ergebnisse:")
print(f"  Ortsteil-basiert: {ortsteil_matches:,} Matches")
print(f"  Street-basiert: {street_matches:,} Matches")
print(f"  Total mit PLZ: {total_with_plz:,}/{len(df_normalized):,} ({total_with_plz/len(df_normalized)*100:.1f}%)")

# Zeige PLZ-Verteilung
if total_with_plz > 0:
    print(f"\nTop 10 PLZ im 2018-2019 Dataset:")
    plz_counts = df_normalized['plz'].value_counts().head(10)
    for plz, count in plz_counts.items():
        print(f"  {plz}: {count:,} Einträge")

print(f"\n✅ PLZ-Extraktion für 2018-2019 Dataset abgeschlossen!")

PLZ-EXTRAKTION FÜR 2018-2019 DATASET
df_normalized Spalten: ['price', 'size', 'district', 'rooms', 'year', 'dataset_id', 'source', 'street', 'floor', 'typeOfFlat', 'yearConstructed', 'totalRent', 'plz']
df_raw Spalten: ['regio3', 'street', 'livingSpace', 'baseRent', 'totalRent', 'noRooms', 'floor', 'typeOfFlat', 'yearConstructed']
❌ Längen-Mismatch: df_normalized=10387, df_raw=10406

=== STRATEGIE 1: ORTSTEIL-BASIERTE PLZ-EXTRAKTION ===
Ortsteil-PLZ-Mapping erstellt: 91 Einträge
Beispiele:
  Halensee → 10713
  Hakenfelde → 13587
  Lichterfelde → 12209
  Charlottenburg → 14055
  Marienfelde → 12307

=== STRATEGIE 2: STREET-BASIERTE PLZ-EXTRAKTION ===
Ortsteil-PLZ-Mapping erstellt: 91 Einträge
Beispiele:
  Halensee → 10713
  Hakenfelde → 13587
  Lichterfelde → 12209
  Charlottenburg → 14055
  Marienfelde → 12307

=== STRATEGIE 2: STREET-BASIERTE PLZ-EXTRAKTION ===
Street-PLZ-Mapping erstellt: 9479 Einträge
Beispiele:
  Aachener Straße → 10713
  Aalemannufer → 13587
  Aarauer Straße → 122

In [18]:
# ===================================================================
# ERWEITERTE PLZ-EXTRAKTION FÜR 2018-2019 DATASET
# ===================================================================
print("\n🔍 ERWEITERTE PLZ-EXTRAKTION FÜR 2018-2019")
print("=" * 60)

def extract_plz_advanced_2018_2019(street, district, ortsteil_mapping=None, street_mapping=None):
    """
    Erweiterte PLZ-Extraktion für 2018-2019 Dataset mit mehreren Fallback-Strategien
    
    Args:
        street: Straßenname
        district: Bezirk
        ortsteil_mapping: Ortsteil-zu-PLZ-Mapping
        street_mapping: Street-zu-PLZ-Mapping
    
    Returns:
        str: PLZ oder None
    """
    # Strategie 1: Street-basierte PLZ-Extraktion (falls vorhanden)
    if pd.notna(street) and street_mapping and street in street_mapping:
        return str(street_mapping[street])
    
    # Strategie 2: Bezirk-zu-PLZ-Mapping (erweitert)
    if pd.notna(district) and ortsteil_mapping:
        district_str = str(district).strip()
        
        # Direkte Bezirks-Zuordnung
        if district_str in ortsteil_mapping:
            return str(ortsteil_mapping[district_str])
        
        # Erweiterte Bezirks-Aliases
        district_aliases = {
            # Hauptbezirke
            'Mitte': 'Mitte',
            'Friedrichshain-Kreuzberg': 'Friedrichshain',
            'Pankow': 'Pankow',
            'Charlottenburg-Wilmersdorf': 'Charlottenburg',
            'Spandau': 'Spandau',
            'Steglitz-Zehlendorf': 'Steglitz',
            'Tempelhof-Schöneberg': 'Tempelhof',
            'Neukölln': 'Neukölln',
            'Treptow-Köpenick': 'Treptow',
            'Marzahn-Hellersdorf': 'Marzahn',
            'Lichtenberg': 'Lichtenberg',
            'Reinickendorf': 'Reinickendorf',
            # Einzelteile zusammengesetzter Bezirke
            'Friedrichshain': 'Friedrichshain',
            'Kreuzberg': 'Kreuzberg',
            'Charlottenburg': 'Charlottenburg',
            'Wilmersdorf': 'Wilmersdorf',
            'Steglitz': 'Steglitz',
            'Zehlendorf': 'Zehlendorf',
            'Tempelhof': 'Tempelhof',
            'Schöneberg': 'Schöneberg',
            'Treptow': 'Treptow',
            'Köpenick': 'Köpenick',
            'Marzahn': 'Marzahn',
            'Hellersdorf': 'Hellersdorf'
        }
        
        # Prüfe Aliases
        for alias, canonical in district_aliases.items():
            if alias.lower() in district_str.lower():
                if canonical in ortsteil_mapping:
                    return str(ortsteil_mapping[canonical])
    
    # Strategie 3: Häufige Straßenname-Patterns
    if pd.notna(street):
        street_str = str(street).strip()
        
        # Bekannte Straßen-zu-PLZ-Mapping (häufigste Berliner Straßen)
        common_streets = {
            'Unter den Linden': '10117',
            'Alexanderplatz': '10178',
            'Potsdamer Platz': '10785',
            'Kurfürstendamm': '10719',
            'Friedrichstraße': '10117',
            'Hackescher Markt': '10178',
            'Warschauer Straße': '10243',
            'Boxhagener Straße': '10245',
            'Kastanienallee': '10435',
            'Oranienstraße': '10999',
            'Bergmannstraße': '10961',
            'Savignyplatz': '10623',
            'Rosenthaler Straße': '10119',
            'Torstraße': '10119',
            'Invalidenstraße': '10115',
            'Chausseestraße': '10115',
            'Brunnenstraße': '10119',
            'Bernauer Straße': '10119',
            'Prenzlauer Allee': '10405',
            'Karl-Marx-Allee': '10243',
            'Frankfurter Allee': '10247',
            'Sonnenallee': '12047',
            'Hermannstraße': '12049',
            'Kantstraße': '10623',
            'Wilmersdorfer Straße': '10627',
            'Uhlandstraße': '10623',
            'Ku\'damm': '10719',
            'Tauentzienstraße': '10789',
            'Nollendorfplatz': '10777',
            'Wittenbergplatz': '10789'
        }
        
        # Prüfe auf bekannte Straßen (auch Teilstrings)
        for known_street, plz in common_streets.items():
            if known_street.lower() in street_str.lower():
                return plz
    
    return None

# Teste die erweiterte PLZ-Extraktion
print("🧪 TESTE ERWEITERTE PLZ-EXTRAKTION")
print("=" * 50)

test_cases = [
    ("Unter den Linden", "Mitte"),
    ("Alexanderplatz", "Mitte"),
    ("Boxhagener Straße", "Friedrichshain"),
    ("Kurfürstendamm", "Charlottenburg"),
    ("Sonnenallee", "Neukölln"),
    ("Hauptstraße", "Steglitz"),
    ("Müllerstraße", "Wedding")
]

print("Test der erweiterten PLZ-Extraktion:")
for street, district in test_cases:
    plz = extract_plz_advanced_2018_2019(street, district, ortsteil_plz_mapping, street_plz_mapping)
    print(f"   '{street}' in '{district}' → PLZ: {plz}")

# Erstelle erweiterte PLZ-Extraktion für alle Daten
print("\n🔄 WENDE ERWEITERTE PLZ-EXTRAKTION AN")
print("=" * 50)

# Verwende die verfügbaren Spalten
available_columns = df_normalized.columns.tolist()
print(f"Verfügbare Spalten: {available_columns}")

# Wende erweiterte PLZ-Extraktion an
df_normalized['plz_advanced'] = df_normalized.apply(
    lambda row: extract_plz_advanced_2018_2019(
        row.get('street', None),
        row.get('district', None),
        ortsteil_plz_mapping,
        street_plz_mapping
    ),
    axis=1
)

# Kombiniere alte und neue PLZ-Extraktion
df_normalized['plz_combined'] = df_normalized['plz_advanced'].fillna(df_normalized['plz'])

# Ergebnisse der erweiterten PLZ-Extraktion
plz_advanced_found = df_normalized['plz_advanced'].notna().sum()
plz_combined_found = df_normalized['plz_combined'].notna().sum()
total_rows = len(df_normalized)

print(f"✅ Erweiterte PLZ-Extraktion Ergebnisse:")
print(f"   📊 Nur erweiterte Methode: {plz_advanced_found:,} von {total_rows:,} ({plz_advanced_found/total_rows*100:.1f}%)")
print(f"   📊 Kombiniert (alt + neu): {plz_combined_found:,} von {total_rows:,} ({plz_combined_found/total_rows*100:.1f}%)")

# Verbesserung berechnen
old_plz_count = df_normalized['plz'].notna().sum()
improvement = plz_combined_found - old_plz_count
improvement_pct = (improvement / old_plz_count) * 100 if old_plz_count > 0 else 0

print(f"   📈 Verbesserung: +{improvement:,} PLZ ({improvement_pct:.1f}% mehr)")

if plz_combined_found / total_rows >= 0.8:
    print(f"   🎉 ZIEL ERREICHT: >80% PLZ-Abdeckung!")
elif improvement > 0:
    print(f"   📈 VERBESSERUNG: Erhöhte PLZ-Abdeckung")
else:
    print(f"   ⚠️  KEINE VERBESSERUNG: Alternative Strategie erforderlich")

# Aktualisiere die PLZ-Spalte mit dem kombinierten Ergebnis
df_normalized['plz'] = df_normalized['plz_combined']

# Zeige verbesserte PLZ-Verteilung
if plz_combined_found > 0:
    print(f"\n📋 Verbesserte PLZ-Verteilung (Top 10):")
    plz_counts_new = df_normalized['plz'].value_counts().head(10)
    for plz, count in plz_counts_new.items():
        print(f"   {plz}: {count:,} Einträge")

# Bereinige temporäre Spalten
df_normalized = df_normalized.drop(['plz_advanced', 'plz_combined'], axis=1)

print(f"\n✅ Erweiterte PLZ-Extraktion für 2018-2019 Dataset abgeschlossen!")


🔍 ERWEITERTE PLZ-EXTRAKTION FÜR 2018-2019
🧪 TESTE ERWEITERTE PLZ-EXTRAKTION
Test der erweiterten PLZ-Extraktion:
   'Unter den Linden' in 'Mitte' → PLZ: 10117
   'Alexanderplatz' in 'Mitte' → PLZ: 10178
   'Boxhagener Straße' in 'Friedrichshain' → PLZ: 10245
   'Kurfürstendamm' in 'Charlottenburg' → PLZ: 10719
   'Sonnenallee' in 'Neukölln' → PLZ: 12437
   'Hauptstraße' in 'Steglitz' → PLZ: 13158
   'Müllerstraße' in 'Wedding' → PLZ: 13353

🔄 WENDE ERWEITERTE PLZ-EXTRAKTION AN
Verfügbare Spalten: ['price', 'size', 'district', 'rooms', 'year', 'dataset_id', 'source', 'street', 'floor', 'typeOfFlat', 'yearConstructed', 'totalRent', 'plz']
✅ Erweiterte PLZ-Extraktion Ergebnisse:
   📊 Nur erweiterte Methode: 9,505 von 10,387 (91.5%)
   📊 Kombiniert (alt + neu): 9,505 von 10,387 (91.5%)
   📈 Verbesserung: +7,745 PLZ (440.1% mehr)
   🎉 ZIEL ERREICHT: >80% PLZ-Abdeckung!

📋 Verbesserte PLZ-Verteilung (Top 10):
   10179: 762 Einträge
   10557: 721 Einträge
   14055: 567 Einträge
   12555: 498

## 8. Export des finalen angereicherten Datasets

In [15]:
print("="*60)
print("EXPORT FINALES ANGEREICHERTES DATASET")
print("="*60)

# Export
output_file_enriched = 'data/processed/dataset_2018_2019_enriched.csv'
df_enriched.to_csv(output_file_enriched, index=False)

print(f"✅ Finales angereichertes Dataset exportiert: {output_file_enriched}")
print(f"Dateigröße: {len(df_enriched):,} Zeilen x {len(df_enriched.columns)} Spalten")

# Validierung durch Wiedereinlesen
test_df_enriched = pd.read_csv(output_file_enriched)
print(f"✅ Export-Validierung erfolgreich: {len(test_df_enriched):,} Zeilen geladen")

EXPORT FINALES ANGEREICHERTES DATASET
✅ Finales angereichertes Dataset exportiert: data/processed/dataset_2018_2019_enriched.csv
Dateigröße: 10,387 Zeilen x 15 Spalten
✅ Export-Validierung erfolgreich: 10,387 Zeilen geladen


In [14]:
# ===================================================================
# KRITISCHE PLZ-DATENTYP-REPARATUR VOR EXPORT
# ===================================================================
print("\n🔧 KRITISCHE PLZ-DATENTYP-REPARATUR VOR EXPORT")
print("=" * 60)

# Stelle sicher, dass PLZ als STRING gespeichert wird (nicht als Integer/Float)
# Dies ist kritisch für den späteren Join in 04_Combine_Datasets.ipynb
print("PLZ-Datentyp vor Reparatur:", df_enriched['plz'].dtype)
print("PLZ-Beispiele vor Reparatur:", df_enriched['plz'].dropna().head(3).tolist())

# Konvertiere PLZ zu String
df_enriched['plz'] = df_enriched['plz'].apply(
    lambda x: str(int(x)) if pd.notna(x) else None
)

print("PLZ-Datentyp nach Reparatur:", df_enriched['plz'].dtype)  
print("PLZ-Beispiele nach Reparatur:", df_enriched['plz'].dropna().head(3).tolist())

print("✅ PLZ-Datentyp-Reparatur abgeschlossen!")
print("   ➡️  PLZ wird jetzt als String gespeichert für korrekten Join")


🔧 KRITISCHE PLZ-DATENTYP-REPARATUR VOR EXPORT
PLZ-Datentyp vor Reparatur: object
PLZ-Beispiele vor Reparatur: [np.int64(13591), np.int64(12527), np.int64(13053)]
PLZ-Datentyp nach Reparatur: object
PLZ-Beispiele nach Reparatur: ['13591', '12527', '13053']
✅ PLZ-Datentyp-Reparatur abgeschlossen!
   ➡️  PLZ wird jetzt als String gespeichert für korrekten Join
